In [1]:
import joblib
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np
import cv2


In [2]:
# Load the model

loaded_model = load_model('efficientnetb0-ChestXRay.h5')

In [3]:
loaded_model

In [4]:
# Create prediction function
def pred_img(img_path):
    # Preprocess the image
    image = Image.open(img_path)
    image = image.resize((224, 224))
    print(image)
    image = np.array(image) / 255.0
    image = np.expand_dims(image, axis=0)  # Add batch dimension

    # Pass the image through the model
    pred = loaded_model.predict(image)

    return pred

In [5]:
pred_img('test2.jpg')

<PIL.Image.Image image mode=RGB size=224x224 at 0x13549558C40>
1/1 [==============================] - 2s 2s/step


array([[0.00503196, 0.994968  ]], dtype=float32)

In [6]:
# Test the model
img_path = 'test2.jpg'
predictions = pred_img(img_path)
class_labels = ['Normal', 'Pneumonia']  # Classes names
predicted_class_index = np.argmax(predictions[0])   # get predicted class index
predicted_class_label = class_labels[predicted_class_index]   # get predicted class name
confidence = predictions[0][predicted_class_index]   # get predicted class accuracy

print(f"Predicted class: {predicted_class_label}")
print(f"Confidence: {confidence}")

<PIL.Image.Image image mode=RGB size=224x224 at 0x1354BE9EC80>
1/1 [==============================] - 0s 104ms/step
Predicted class: Pneumonia
Confidence: 0.9949679970741272


# Deployment Model

In [7]:
# !pip install streamlit


In [21]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import streamlit as st
from tensorflow.keras.models import load_model


@st.cache(allow_output_mutation=True)
def load_model():
  model=tf.keras.models.load_model('efficientnetb0-ChestXRay.h5')
  return model
with st.spinner('Model is being loaded..'):
  model=load_model()

st.write("""
         # X-Ray Chest Classification
         """
         )

file = st.file_uploader("Please upload an image", type=["jpg", "png"])
import cv2
from PIL import Image, ImageOps
import numpy as np
st.set_option('deprecation.showfileUploaderEncoding', False)
class_labels = ['Normal', 'Pneumonia']  # Classes names
def import_and_predict(image_data, model):
    
        size = (224,224)    
        image = ImageOps.fit(image_data, size, Image.ANTIALIAS)
        image = np.asarray(image)
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #img_resize = (cv2.resize(img, dsize=(75, 75),    interpolation=cv2.INTER_CUBIC))/255.
        
        img_reshape = img[np.newaxis,...]
    
        prediction = model.predict(img_reshape)
        
        return prediction
if file is None:
    st.text("Please upload an image file")
else:
    image = Image.open(file)
    st.image(image, use_column_width=True)
    predictions = import_and_predict(image, model)
    score = tf.nn.softmax(predictions[0])
    st.write(predictions)
    st.write(score)
    st.write("This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_labels[np.argmax(score)], 100 * np.max(score)))
    print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_labels[np.argmax(score)], 100 * np.max(score))
)



Overwriting app.py
